# 데이터 전처리

## 라이브러리 선언

In [1]:
import numpy as np
import pandas as pd

## 함수 선언

In [7]:
def make_seoul_delivery(origin_df: pd.DataFrame, start_date=None, end_date=None) -> pd.DataFrame:
    """
    원본 데이터프레임에서 '서울특별시' 부분을 추출하는 함수
    origin_df: 원본 데이터프레임
    start_date: 맨 윗 열의 '일자' 인덱스 (기본값: 1월 1일)
    end_date: 맨 아랫 열의 '일자' 인덱스 (기본값: 12월 31일)
    seoul_df: '행정구역'에서 '서울특별시'에 해당하는 행만 걸러낸 데이터프레임, 함수 반환값
    """
    
    seoul_df = origin_df[ origin_df['행정구역'] == '서울특별시' ]
    seoul_df = pd.pivot_table(seoul_df, index=['행정구역', '일자', '분류'], aggfunc=np.sum)
    seoul_df = seoul_df.reset_index()
    del seoul_df['행정구역']
    del seoul_df['시간']

    seoul_df = detach_year(seoul_df)
    seoul_df = seoul_df.set_index('일자')
    
    return seoul_df.loc[start_date : end_date]

In [3]:
def detach_year(origin_df: pd.DataFrame) -> pd.DataFrame:
    """
    원본 데이터프레임의 '일자' 열에 속한 데이터에서 연도와 월일을 분리
    origin_df: 원본 데이터프레임
    detached_df: 원본 데이터프레임의 사본 (원본 데이터프레임 변형 방지 목적), 함수 반환값
    """

    detached_df = origin_df.copy()

    for row in detached_df.iterrows():
        date = row[1]['일자']
        detached_df.at[row[0], '일자'] = date[5:]

    return detached_df

In [13]:
def count_by_category(origin_df: pd.DataFrame, sum_df: pd.DataFrame, categories: list) -> pd.DataFrame:
    """
    음식 '분류' 별로 '주문횟수'를 추출해서 새로운 열을 만드는 함수
    origin_df: 원본 데이터프레임
    sum_df: '총주문횟수' 열을 가지고 있는 데이터프레임
    categories: 음식 분류 목록
    counting_df: sum_df의 사본 (원본 데이터인 sum_df의 변형 방지 목적), 함수 반환값
    each_delivery: 특정 음식 '분류'에 해당하는 행만 추출한 데이터프레임
    """

    counting_df = sum_df.copy()

    for category in categories:
        each_delivery = origin_df[ origin_df['분류'] == category ].copy()
        del each_delivery['분류']
        each_delivery.rename(columns={'주문횟수':category}, inplace=True)
        counting_df = counting_df.join(each_delivery)

    counting_df = counting_df.fillna(0)

    return counting_df

In [5]:
def rename_kor_to_eng(origin_df: pd.DataFrame) -> pd.DataFrame:
    """
    한글로 된 열 이름을 영어로 변환하는 함수
    origin_df: 원본 데이터프레임
    renamed_df: 원본 데이터프레임의 사본 (원본 데이터프레임 변형 방지 목적), 함수 반환값
    en_dictionary: 한영 변환할 때 문자열을 매칭시키기 위해 사용하는 딕셔너리
    en_categories: 영어로 변환된 열 이름을 정렬하기 위해 사용하는 리스트
    """

    renamed_df = origin_df.copy()

    en_dictionary = dict()
    en_dictionary['총주문횟수'] = 'sum'
    en_dictionary['도시락'] = 'dosirak'
    en_dictionary['돈까스/일식'] = 'jpfood'
    en_dictionary['분식'] = 'bunsik'
    en_dictionary['심부름'] = 'simburum'
    en_dictionary['아시안/양식'] = 'wsfood'
    en_dictionary['족발/보쌈'] = 'bossam'
    en_dictionary['중식'] = 'cnfood'
    en_dictionary['찜탕'] = 'tang'
    en_dictionary['치킨'] = 'chicken'
    en_dictionary['카페/디저트'] = 'dessert'
    en_dictionary['패스트푸드'] = 'fastfood'
    en_dictionary['피자'] = 'pizza'
    en_dictionary['한식'] = 'krfood'
    en_dictionary['회'] = 'sashimi'

    renamed_df.rename(columns=en_dictionary, inplace=True)
    renamed_df.index.name = 'date'

    en_categories = list(en_dictionary.values())
    # en_categories = renamed_df_en.columns.tolist()
    en_categories = [en_categories[0]] + sorted(en_categories[1:])
    renamed_df = renamed_df.reindex(columns=en_categories)

    return renamed_df

## 한글 데이터 전처리

In [49]:
year = input()

In [41]:
original_delivery = pd.read_csv(f'original_data/KT_DLVR_{year}.csv', encoding='utf-8')

In [42]:
seoul_delivery = make_seoul_delivery(original_delivery)

In [43]:
sum_delivery = pd.pivot_table(seoul_delivery, index=['일자'], aggfunc=np.sum)
sum_delivery.rename(columns={'주문횟수':'총주문횟수'}, inplace=True)
delivery_categories = sorted(list(set(seoul_delivery['분류'])))

In [44]:
counted_delivery = count_by_category(seoul_delivery, sum_delivery, delivery_categories)

In [45]:
del counted_delivery['배달전문업체']
del counted_delivery['야식']

In [46]:
# counted_delivery.to_csv(f'delivery_{year}.csv')
counted_delivery

,총주문횟수,도시락,돈까스/일식,분식,심부름,아시안/양식,족발/보쌈,중식,찜탕,치킨,카페/디저트,패스트푸드,피자,한식,회
일자,,,,,,,,,,,,,,,
01-01,8786,28.0,465,1102,9.0,54.0,683.0,209.0,298.0,2218,884,720,393.0,861.0,227.0
01-02,9694,60.0,590,1113,19.0,114.0,661.0,217.0,315.0,2087,1539,904,394.0,794.0,215.0
01-03,9142,18.0,524,1025,6.0,93.0,592.0,284.0,329.0,1777,1343,975,455.0,878.0,112.0
01-04,6402,165.0,499,748,40.0,105.0,348.0,117.0,204.0,1383,697,709,184.0,614.0,76.0
01-05,6807,164.0,521,867,29.0,75.0,469.0,155.0,216.0,1383,638,682,258.0,685.0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
07-27,2394,0.0,849,251,0.0,16.0,40.0,20.0,93.0,355,283,252,8.0,160.0,6.0
07-28,2529,2.0,892,248,0.0,19.0,36.0,17.0,81.0,404,271,323,22.0,164.0,4.0
07-29,2488,0.0,958,243,0.0,23.0,34.0,20.0,66.0,313,270,301,16.0,162.0,1.0


## 데이터 한영 변환

In [47]:
counted_delivery_en = rename_kor_to_eng(counted_delivery)

In [48]:
# counted_delivery_en.to_csv(f'delivery_{year}.csv')
counted_delivery_en

,sum,bossam,bunsik,chicken,cnfood,dessert,dosirak,fastfood,jpfood,krfood,pizza,sashimi,simburum,tang,wsfood
date,,,,,,,,,,,,,,,
01-01,8786,683.0,1102,2218,209.0,884,28.0,720,465,861.0,393.0,227.0,9.0,298.0,54.0
01-02,9694,661.0,1113,2087,217.0,1539,60.0,904,590,794.0,394.0,215.0,19.0,315.0,114.0
01-03,9142,592.0,1025,1777,284.0,1343,18.0,975,524,878.0,455.0,112.0,6.0,329.0,93.0
01-04,6402,348.0,748,1383,117.0,697,165.0,709,499,614.0,184.0,76.0,40.0,204.0,105.0
01-05,6807,469.0,867,1383,155.0,638,164.0,682,521,685.0,258.0,75.0,29.0,216.0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
07-27,2394,40.0,251,355,20.0,283,0.0,252,849,160.0,8.0,6.0,0.0,93.0,16.0
07-28,2529,36.0,248,404,17.0,271,2.0,323,892,164.0,22.0,4.0,0.0,81.0,19.0
07-29,2488,34.0,243,313,20.0,270,0.0,301,958,162.0,16.0,1.0,0.0,66.0,23.0


## 데이터에서 월-일 분리

In [50]:
def detach_month(origin_df: pd.DataFrame) -> pd.DataFrame:
    """
    원본 데이터프레임의 '일자' 열에 속한 데이터에서 월과 일을 분리
    origin_df: 원본 데이터프레임
    detached_df: 원본 데이터프레임의 사본 (원본 데이터프레임 변형 방지 목적), 함수 반환값
    """

    detached_df = origin_df.copy()

    for row in detached_df.iterrows():
        date = row[1]['date']
        detached_df.at[row[0], 'date'] = date[-2:]

    return detached_df

In [51]:
delivery_day = counted_delivery_en.reset_index()
delivery_day = detach_month(delivery_day)

In [52]:
delivery_day

,date,sum,bossam,bunsik,chicken,cnfood,dessert,dosirak,fastfood,jpfood,krfood,pizza,sashimi,simburum,tang,wsfood
0,01,8786,683.0,1102,2218,209.0,884,28.0,720,465,861.0,393.0,227.0,9.0,298.0,54.0
1,02,9694,661.0,1113,2087,217.0,1539,60.0,904,590,794.0,394.0,215.0,19.0,315.0,114.0
2,03,9142,592.0,1025,1777,284.0,1343,18.0,975,524,878.0,455.0,112.0,6.0,329.0,93.0
3,04,6402,348.0,748,1383,117.0,697,165.0,709,499,614.0,184.0,76.0,40.0,204.0,105.0
4,05,6807,469.0,867,1383,155.0,638,164.0,682,521,685.0,258.0,75.0,29.0,216.0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,27,2394,40.0,251,355,20.0,283,0.0,252,849,160.0,8.0,6.0,0.0,93.0,16.0
201,28,2529,36.0,248,404,17.0,271,2.0,323,892,164.0,22.0,4.0,0.0,81.0,19.0
202,29,2488,34.0,243,313,20.0,270,0.0,301,958,162.0,16.0,1.0,0.0,66.0,23.0
203,30,2887,48.0,340,413,24.0,337,2.0,280,1069,140.0,13.0,7.0,0.0,90.0,35.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=aeda0379-ef07-4599-92da-f5608bf4c48d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>